# Make movies from LLC

In [1]:
# Load some useful modules 
import numpy as np
import xarray as xr
import xrft
from xmitgcm import llcreader
from matplotlib import pyplot as plt
from xmovie import Movie
import cmocean.cm as cm

ModuleNotFoundError: No module named 'xmovie'

In [2]:
from intake import open_catalog

cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")

In [3]:
# Get variables from catalog
sst = cat.LLC4320_SST.to_dask()
sss = cat.LLC4320_SSS.to_dask()
ssh = cat.LLC4320_SSH.to_dask()
u = cat.LLC4320_SSU.to_dask()
v = cat.LLC4320_SSV.to_dask()

In [4]:
coords = (cat.LLC4320_grid.to_dask()).sel(face=1)

In [5]:
ds = xr.merge([ssh.sel(face=1), sst.sel(face=1), sss.sel(face=1), 
               u.sel(face=1), v.sel(face=1)])

In [6]:
import xgcm
grid = xgcm.Grid(coords.drop(['k', 'k_p1']), periodic=None)

In [7]:
from fastjmd95 import rho
ds['SSD'] = xr.apply_ufunc(rho, 
                        ds.SSS, ds.SST, 0, 
                        dask='parallelized', output_dtypes=[float,]).rename('SSD')

import gsw
ds['Spice'] = xr.apply_ufunc(gsw.spiciness0, 
                        ds.SSS, ds.SST,  
                        dask='parallelized', output_dtypes=[float,]).rename('Spice')

In [8]:
glid659 = xr.load_dataset('./glider_data/659_locs.nc')
glid660 = xr.load_dataset('./glider_data/660_locs.nc')

In [9]:
sel_XC = np.logical_and(coords.XC>25, coords.XC<40 ) 
sel_XG = np.logical_and(coords.XG>25, coords.XG<40 )
sel_YC = np.logical_and(coords.YC>-60, coords.YC<-45 )
sel_YG = np.logical_and(coords.YG>-60, coords.YG<-45 )

In [10]:
XC = coords.XC.where(sel_XC & sel_YC, drop=True)
YC = coords.YC.where(sel_XC & sel_YC, drop=True)
XCmean = XC.mean(['j'])
YCmean = YC.mean(['i'])

Xmax = XC.max(['j'])
Xmin = XC.min(['j'])
Ymax = YC.max(['i'])
Ymin = YC.min(['i'])

In [11]:
XG = coords.XG.where(sel_XG & sel_YG, drop=True)
YG = coords.YG.where(sel_XG & sel_YG, drop=True)
XGmean = XG.mean(['j_g'])
YGmean = YG.mean(['i_g'])

## Movie of Surface Tracers

In [13]:
SSH_sogos = ds.Eta.where(sel_XC & sel_YC, drop=True)
SST_sogos = ds.SST.where(sel_XC & sel_YC, drop=True)
SSS_sogos = ds.SSS.where(sel_XC & sel_YC, drop=True)
SSD_sogos = ds.SSD.where(sel_XC & sel_YC, drop=True)
Spice_sogos = ds.Spice.where(sel_XC & sel_YC, drop=True)

In [14]:
ds_sogos = xr.merge([SST_sogos, SSS_sogos, SSD_sogos, Spice_sogos])

In [15]:
def custom_plotfunc(ds_sogos, fig, tt):
    
    ds_sogos = ds_sogos.isel(i=slice(0,-1,4),j=slice(0,-1,4))
    XC = ds_sogos.XC
    YC = ds_sogos.YC 
    
    ax = fig.subplots(2, 2)

    ax1= ax[0,0]
    ax2= ax[0,1]
    ax3= ax[1,0]
    ax4= ax[1,1]

    p1 = ax1.pcolormesh(XC, YC, ds_sogos.SST.isel(time=tt), cmap=cm.thermal, vmin=-1, vmax=9)
    ax1.contour(XC, YC, ds_sogos.SST.isel(time=tt), levels=[2.5], colors='k')
    ax1.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar1 = fig.colorbar(p1 , ax=ax1)
    cbar1.ax.set_ylabel('Temp')
    ax1.set_aspect(1.25)
    ax1.set_title(str((ds_sogos.time.isel(time=tt)).dt.strftime("%b %d, %y").values))

    p2 = ax2.pcolormesh(XC, YC, ds_sogos.SSS.isel(time=tt), cmap=cm.haline, vmin=33.8, vmax=34.7)
    ax2.contour(XC, YC, ds_sogos.SSS.isel(time=tt), levels=[33.9], colors='k')
    ax2.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar2 = fig.colorbar(p2 , ax=ax2)
    cbar2.ax.set_ylabel('Salt')
    ax2.set_aspect(1.25)
    ax2.set_title(str((ds_sogos.time.isel(time=tt)).dt.season.values))

    p3 = ax3.pcolormesh(XC, YC, ds_sogos.SSD.isel(time=tt), cmap=cm.dense, vmin=1026.5, vmax=1027.4)
    ax3.contour(XC, YC, ds_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k')
    ax3.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar3 = fig.colorbar(p3, ax=ax3)
    cbar3.ax.set_ylabel('Density')
    ax3.set_aspect(1.25)
    #ax3.set_title(str((SST_sogos.time.isel(time=tt)).dt.season.values))

    p4 = ax4.pcolormesh(XC, YC, ds_sogos.Spice.isel(time=tt), cmap=cm.deep, vmin=-1, vmax=0.8)
    ax4.contour(XC, YC, ds_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k')
    ax4.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar4 = fig.colorbar(p4, ax=ax4)
    cbar4.ax.set_ylabel('Spice')
    ax4.set_aspect(1.25)

    fig.tight_layout()


In [16]:
mov_custom = Movie(ds_sogos.isel(time=slice(0,-1,24)), 
                   custom_plotfunc, 
                   input_check=False)

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
mov_custom.save('movie_TSDS/movie_TSDS.mp4'
                ,progress=True,remove_frames=False,
               remove_movie=False, start_frame=369,)

movie_TSDS/movie_TSDS.mp4



Movie created at movie_TSDS.mp4


## Movie of surface kinematics

In [12]:
KE = 0.5*(grid.interp(ds.U,'X',boundary='extend')**2 + grid.interp(ds.V,'Y',boundary='extend')**2) 

zeta = (-grid.diff(ds.U * coords.dxC, 'Y', boundary='extend') + grid.diff(ds.V * coords.dyC, 'X', boundary='extend'))/coords.rAz
zeta = grid.interp(grid.interp(zeta, 'X', boundary='extend'), 'Y', boundary='extend')

strain1 = (grid.diff(ds.U * coords.dyG, 'X', boundary='extend') - grid.diff(ds.V * coords.dxG, 'Y',boundary='extend')) / coords.rA
strain2 = (grid.diff(ds.U * coords.dxC, 'Y', boundary='extend') + grid.diff(ds.V * coords.dyC, 'X', boundary='extend'))/coords.rAz
strain2 = grid.interp(grid.interp(strain2, 'X', boundary='extend'), 'Y', boundary='extend')
strain = (strain1**2 + strain2**2)**0.5

gradD = (grid.interp(grid.diff(ds.SSD,'X',boundary='extend')/coords.dxC, 'X', boundary='extend')**2 +
         grid.interp(grid.diff(ds.SSD,'Y',boundary='extend')/coords.dyC, 'Y', boundary='extend')**2)**0.5

In [13]:
zeta_sogos = zeta.where(sel_XC & sel_YC, drop=True)
strain_sogos = strain.where(sel_XC & sel_YC, drop=True)
gradD_sogos = gradD.where(sel_XC & sel_YC, drop=True)
KE_sogos = KE.where(sel_XC & sel_YC, drop=True)
SSD_sogos = ds.SSD.where(sel_XC & sel_YC, drop=True)

In [14]:
f = 2*(2*np.pi/24/3600)*np.sin(-55*np.pi/360)

ds_kinem_sogos = xr.merge([zeta_sogos.rename('Vorticity')/f,
                           strain_sogos.rename('Strain')/(-f), 
                           gradD_sogos.rename('rho_grad'),
                           KE_sogos.rename('KE'),
                          SSD_sogos])
# f=2*omega*sin(theta)


In [15]:
def custom_plotfunc2(ds_kinem_sogos, fig, tt):
    
    ds_kinem_sogos = ds_kinem_sogos.isel(i=slice(0,-1,1),j=slice(0,-1,1))

    XC = ds_kinem_sogos.XC
    YC = ds_kinem_sogos.YC 

    ax = fig.subplots(2, 2)

    ax1= ax[0,0]
    ax2= ax[0,1]
    ax3= ax[1,0]
    ax4= ax[1,1]

    p1 = ax1.pcolormesh(XC, YC, ds_kinem_sogos.KE.isel(time=tt), cmap=cm.speed_r, vmin=0, vmax=0.8)
    ax1.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax1.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar1 = fig.colorbar(p1 , ax=ax1)
    cbar1.ax.set_ylabel('KE (m/s)')
    ax1.set_aspect(1.25)
    ax1.set_title(str((ds_kinem_sogos.time.isel(time=tt)).dt.strftime("%b %d, %y").values))

    p2 = ax2.pcolormesh(XC, YC, ds_kinem_sogos.Vorticity.isel(time=tt), cmap=cm.curl, vmin=-1, vmax=1)
    ax2.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax2.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar2 = fig.colorbar(p2 , ax=ax2)
    cbar2.ax.set_ylabel('Vorticity/f')
    ax2.set_aspect(1.25)
    ax2.set_title(str((ds_kinem_sogos.time.isel(time=tt)).dt.season.values))

    p3 = ax3.pcolormesh(XC, YC, ds_kinem_sogos.Strain.isel(time=tt), cmap=cm.amp, vmin=0, vmax=1)
    ax3.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax3.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar3 = fig.colorbar(p3, ax=ax3)
    cbar3.ax.set_ylabel('Strain/f')
    ax3.set_aspect(1.25)

    p4 = ax4.pcolormesh(XC, YC, ds_kinem_sogos.rho_grad.isel(time=tt), cmap=cm.ice, vmin=0.1e-5, vmax= 3.5e-5)
    ax4.contour(XC, YC, ds_kinem_sogos.SSD.isel(time=tt), levels=[1027.1], colors='k',linewidths=1)
    ax4.plot(glid659.longitude, glid659.latitude,color='white', linewidth=2)
    cbar4 = fig.colorbar(p4, ax=ax4)
    cbar4.ax.set_ylabel(r'$|\nabla \rho|$')
    ax4.set_aspect(1.25)

    fig.tight_layout()

In [16]:
import warnings
warnings.filterwarnings("ignore")

In [17]:
mov_custom_2 = Movie(ds_kinem_sogos.isel(time=slice(0,-1,24)), 
                   custom_plotfunc2, 
                   input_check=False)

In [18]:
mov_custom_2.save('movie_KZSG/movie_KZSG.mp4'
                ,progress=True,remove_frames=False,
                remove_movie=False, start_frame=113)

movie_KZSG/movie_KZSG.mp4



Movie created at movie_KZSG.mp4


In [20]:
import os
dirname = os.path.dirname('movie_KZSG/movie_KZSG.mp4')

In [21]:
dirname

'movie_KZSG'

In [25]:
#from xmovie.core import frame_save

def frame_save(fig, frame, odir=None, frame_pattern="frame_%05d.png", dpi=100):
    fig.savefig(
        os.path.join(odir, frame_pattern % (frame)),
        dpi=dpi,
        facecolor=fig.get_facecolor(),
        transparent=True,
    )
    # I am trying everything to *wipe* this figure, hoping that it could
    # help with the dask glitches I experienced earlier.
    # TBD if this is all needed...how this might affect performance.
    plt.close('all')
    #del fig
    #gc.collect(2)


In [26]:
%matplotlib notebook

In [27]:
for fi in range(67,75):
    fig, ax, pp = mov_custom_2.render_frame(fi)
    frame_save(fig, fi, odir=dirname, frame_pattern=mov_custom_2.frame_pattern, 
               dpi=mov_custom_2.dpi )

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Scrap code

In [ ]:
import os
import glob
import re

files=glob.glob("movie_TSDS/frame_*.png")
files.sort(key=lambda f: int(re.sub('\D', '', f)))
int(re.sub('\D', '', files[-1]))